In [1]:
import pandas as pd

# Step 1: Load the wide-format test environmental dataset
test_file_path = "C:/Users/emman/Downloads/PM2.5_Pollution_Data-Public/Scripts/Environmental_Data/Training_and_Testing_Dataset/Imputed_test_Environmental_Data.csv"  
test_df = pd.read_csv(test_file_path)

# Step 2: Calculate average PM2.5 values
average_pm25_total = test_df['PM2.5_Total'].mean()
average_pm25_anthro = test_df['PM2.5_Anthropogenic'].mean()
average_pm25_nonanthro = test_df['PM2.5_Non_Anthropogenic'].mean()

# Step 3: Display results
print("Average PM2.5 Total:", average_pm25_total)
print("Average PM2.5 Anthropogenic:", average_pm25_anthro)
print("Average PM2.5 Non-Anthropogenic:", average_pm25_nonanthro)


Average PM2.5 Total: 7.242716974789917
Average PM2.5 Anthropogenic: 6.714745714285715
Average PM2.5 Non-Anthropogenic: 0.5279763025210084


In [3]:
import pandas as pd

# Step 1: Load your PRS data with the correct separator
prs_file = "C:/Users/emman/Downloads/PM2.5_Pollution_Data-Public/Scripts/Genetic_Data/New_Genetic_Data/genetic_data_with_PRS.csv"
prs_df = pd.read_csv(prs_file, sep=',')  # <- changed from \t to ,

# Step 2: Check column names
print(prs_df.columns)

# Step 3: Calculate average PRS score
average_prs = prs_df['PRS'].mean()

# Step 4: Output the result
print("Average PRS Score:", average_prs)


Index(['MarkerName', 'Allele1', 'Allele2', 'Freq1', 'FreqSE', 'MinFreq',
       'MaxFreq', 'Effect', 'StdErr', 'P-value', 'Direction', 'HetISq',
       'HetChiSq', 'HetDf', 'HetPVal', 'oldID', 'CHR', 'BP', 'rsID',
       'chr_name', 'chr_position', 'effect_allele', 'other_allele',
       'effect_weight', 'chr', 'pos', 'ref', 'alt', 'alt_freq', 'PRS'],
      dtype='object')
Average PRS Score: -0.0003159840954274198


1. Filter for Clean Observations
Only keep:

Sex = "Persons"

Age = "18+ yrs" or "All ages" (choose one for consistency)

Non-null Value field

Relevant test period (e.g., years > 2021 if your test set uses data from 2022 onward)

2. Group and Aggregate
Group by Condition

Compute the mean clinical Value for each

In [23]:
import pandas as pd

# Step 1: Load the clinical data
clinical_file = "C:/Users/emman/Downloads/PM2.5_Pollution_Data-Public/Scripts/Clinical_Data/Phenotype_Disease_Conditions/Merged_Clinical_Filtered_Data/merged_clinical_data_V1.csv"
df_clinical = pd.read_csv(clinical_file)

# Step 2: Filter for target years + demographics
filtered_clinical = df_clinical[
    (df_clinical['Age'] == '18+ yrs') &
    (df_clinical['Sex'] == 'Persons') &
    (df_clinical['Time period'].isin(['2022/23', '2023/24']))
]

# Step 3: Conditions of interest
conditions = [
    "CKD: QOF prevalence",
    "Type 2 diabetes: QOF prevalence rate",
    "Asthma: QOF prevalence rate",
    "Alcohol-related cardiovascular disease admissions (Directly standardised rate)"
]

# Step 4: Calculate and print averages
print("Average clinical values (2022/23 & 2023/24):")
for condition in conditions:
    cond_df = filtered_clinical[filtered_clinical['Indicator Name'] == condition]
    avg_val = cond_df['Value'].mean()
    if pd.isna(avg_val):
        print(f"- {condition}: No data available")
    else:
        print(f"- {condition}: {avg_val:.4f}")


Average clinical values (2022/23 & 2023/24):
- CKD: QOF prevalence: 4.2394
- Type 2 diabetes: QOF prevalence rate: No data available
- Asthma: QOF prevalence rate: No data available
- Alcohol-related cardiovascular disease admissions (Directly standardised rate): No data available


In [24]:
import pandas as pd

# === Step 1: Define your average values manually or from previous outputs ===
# Replace these values with your actual results
avg_pm25_total = 8.12                # Example: Replace with your actual value
avg_pm25_anthro = 5.87               # Example: Replace with your actual value
avg_pm25_non_anthro = 2.25           # Example: Replace with your actual value

avg_prs = -0.000316                  # Your calculated average PRS score
avg_ckd = 4.2394                     # Your calculated average CKD clinical value

# === Step 2: Combine into a single dictionary (row of data) ===
combined_data = {
    'PM2.5_Total': avg_pm25_total,
    'PM2.5_Anthropogenic': avg_pm25_anthro,
    'PM2.5_Non_Anthropogenic': avg_pm25_non_anthro,
    'PRS_Average': avg_prs,
    'CKD_QOF_Prevalence': avg_ckd
}

# === Step 3: Convert to DataFrame ===
df_combined = pd.DataFrame([combined_data])

# === Step 4: Preview and Save (optional) ===
print(df_combined)

# Optional: Save to CSV for model input
df_combined.to_csv("Combined_Averages_for_Model_Input.csv", index=False)


   PM2.5_Total  PM2.5_Anthropogenic  PM2.5_Non_Anthropogenic  PRS_Average  \
0         8.12                 5.87                     2.25    -0.000316   

   CKD_QOF_Prevalence  
0              4.2394  


In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# === Step 1: Load the combined averages file ===
df = pd.read_csv("Combined_Averages_for_Model_Input.csv")

# === Step 2: Simulate multiple rows (optional for demo purposes only) ===
# You can remove this if you have a real dataset
df = pd.concat([df] * 50, ignore_index=True)  # Duplicate to make a sample dataset
df['Longevity'] = 80 + df['PRS_Average'] * 1000 - df['PM2.5_Total'] * 0.5 - df['CKD_QOF_Prevalence'] * 0.3  # Simulated target

# === Step 3: Prepare features and target ===
X = df.drop(columns=['Longevity'])  # Features
y = df['Longevity']                 # Target

# === Step 4: Train/test split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Step 5: Train the Random Forest model ===
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# === Step 6: Evaluate ===
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse:.4f}")

# === Step 7: Make a prediction on your actual single-row input ===
single_input = pd.read_csv("Combined_Averages_for_Model_Input.csv")
predicted_outcome = model.predict(single_input)
print(f"\n🧠 Predicted Longevity for your input row: {predicted_outcome[0]:.2f} years")


Mean Squared Error on Test Set: 0.0000

🧠 Predicted Longevity for your input row: 74.35 years


In [26]:
import pandas as pd

avg_total = 8.12
avg_anthro = 5.87  
avg_non_anthro = 2.25  
avg_prs = -0.00032
avg_ckd = 4.2394

# Combine into a single row DataFrame
combined_data = pd.DataFrame([{
    'PM2.5_Total': avg_total,
    'PM2.5_Anthropogenic': avg_anthro,
    'PM2.5_Non_Anthropogenic': avg_non_anthro,
    'PRS': avg_prs,
    'CKD_prevalence': avg_ckd
}])

#Show the data 
print(combined_data)

#  save it
#combined_data.to_csv("combined_test_input.csv", index=False)


   PM2.5_Total  PM2.5_Anthropogenic  PM2.5_Non_Anthropogenic      PRS  \
0         8.12                 5.87                     2.25 -0.00032   

   CKD_prevalence  
0          4.2394  


In [27]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# === Step 1: Load your combined dataset ===
combined_data = pd.DataFrame([{
    'PM2.5_Total': 8.12,
    'PM2.5_Anthropogenic': 5.87,
    'PM2.5_Non_Anthropogenic': 2.25,
    'PRS': -0.00032,
    'CKD_prevalence': 4.2394
}])

# === Step 2: Define features (X) and target (y) ===
X = combined_data[['PM2.5_Total', 'PM2.5_Anthropogenic', 'PM2.5_Non_Anthropogenic', 'PRS']]
y = combined_data['CKD_prevalence']

# === Step 3: Fit a regression model (Random Forest) ===
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# === Step 4: Make prediction on the same input (demo) ===
y_pred = model.predict(X)

# === Step 5: Show the prediction and performance ===
print(f"Predicted CKD prevalence: {y_pred[0]:.4f}")
print(f"Actual CKD prevalence: {y.values[0]:.4f}")

# Optional evaluation (won’t be meaningful with one row, but included for structure)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)
print(f"MSE: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")


Predicted CKD prevalence: 4.2394
Actual CKD prevalence: 4.2394
MSE: 0.0000
R^2 Score: nan


C:\Users\emman\Downloads\Personal_File\University_of_Bradford\Lib\site-packages\sklearn\metrics\_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [2]:
import pandas as pd

scaled_test_data = "C:/Users/emman/Downloads/PM2.5_Pollution_Data-Public/Scripts/Environmental_Data/Training_and_Testing_Dataset/Test_Scaled_PM25.csv"
test_scaled_PM25 = pd.read_csv(scaled_test_data)

# Example: reorder test set columns to match training features
features = ['PM2.5_Total', 'PM2.5_Anthropogenic', 'PM2.5_Non_Anthropogenic']  # adjust if more features
X_test = test_scaled_PM25[features].copy()


In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

# Step 1: Load your datasets
df_wide = pd.read_csv("C:/Users/emman/Downloads/PM2.5_Pollution_Data-Public/Scripts/Environmental_Data/Training_and_Testing_Dataset/Imputed_training_Environmental_Data.csv")
df_wide_test = pd.read_csv("C:/Users/emman/Downloads/PM2.5_Pollution_Data-Public/Scripts/Environmental_Data/Training_and_Testing_Dataset/Imputed_test_Environmental_Data.csv")

# Step 2: Split training and test sets
train_df = df_wide[df_wide['Year'] < 2021].copy()
test_df = df_wide_test[df_wide_test['Year'] >= 2021].copy()  # assuming Year column exists

# Step 3: Prepare features and target for training
X_train = train_df.drop(columns=['PM2.5_Total', 'PM2.5_Anthropogenic', 'PM2.5_Non_Anthropogenic'])
X_train = X_train.select_dtypes(include=['number'])
y_train = train_df['PM2.5_Total']

# Step 4: Prepare test set
X_test = df_wide_test.drop(columns=['PM2.5_Total', 'PM2.5_Anthropogenic', 'PM2.5_Non_Anthropogenic'], errors='ignore')
X_test = X_test.select_dtypes(include=['number'])
# Ensure same columns and order
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Step 5: Train the Random Forest model
model = RandomForestRegressor(n_estimators=500, max_depth=None, random_state=42)
model.fit(X_train, y_train)

# Step 6: Save the trained model
joblib.dump(model, "trained_PM25_model.pkl")

# Step 7: Predict on test data (scaled features)
y_pred = model.predict(X_test)

print(y_pred)


[ 6.9825896   6.9825896   5.3211248   6.6727384   6.6727384   7.2265302
  7.2265302   7.04457433  7.04457433  7.41173913  7.41173913  7.66776825
  7.66776825  7.92875504  7.92875504  7.92875504  7.92875504  6.62541859
  7.64713725  7.64713725  7.7861974   7.7861974   6.7995038   6.7995038
  7.0526014   7.0526014   7.4494076   7.4494076   7.9389178   7.9389178
  7.326836    7.326836    6.7331712   6.7331712   6.2285214   6.2285214
  6.5914026   6.5914026   7.1511002   7.1511002   8.2314674   8.2314674
  7.5272308   7.5272308   7.9830352   7.9830352   7.5314772   7.5314772
  7.5808356   7.5808356   7.5922186   7.5922186   7.8707142   7.8707142
  8.0342364   8.0342364   8.0445898   8.0445898   7.6831978   7.6831978
  7.2108204   7.2108204   7.723882    7.723882    7.447419    7.447419
  6.826668    6.826668    6.942118    6.942118    6.911198    6.911198
  7.4934788   7.4934788   7.0915876   7.0915876   7.3623082   7.3623082
  5.5391396   7.81207     7.81207     7.5323322   7.5323322   7.

In [21]:
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [22]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [23]:
y_pred = rf.predict(X_test)

# See your predictions
print(y_pred)


[ 6.899215    6.899215    5.414331    6.562572    6.562572    7.226446
  7.226446    7.002514    7.002514    7.382085    7.382085    7.71437242
  7.71437242  8.00591175  8.00591175  8.00591175  8.00591175  6.72128967
  7.54230633  7.54230633  7.726879    7.726879    6.7737      6.7737
  7.071007    7.071007    7.450422    7.450422    7.946043    7.946043
  7.349542    7.349542    6.744373    6.744373    6.261764    6.261764
  6.646763    6.646763    7.103151    7.103151    8.220216    8.220216
  7.614637    7.614637    8.032044    8.032044    7.550492    7.550492
  7.586814    7.586814    7.599245    7.599245    7.850558    7.850558
  8.033106    8.033106    8.058312    8.058312    7.692879    7.692879
  7.230437    7.230437    7.799721    7.799721    7.528739    7.528739
  6.859103    6.859103    6.905714    6.905714    6.824923    6.824923
  7.420323    7.420323    7.006743    7.006743    7.306877    7.306877
  5.605251    7.880422    7.880422    7.495762    7.495762    7.784305
  7.

NameError: name 'y_test' is not defined